In [1]:
# Notebook 2: Embedding and Chunking

# Run the first notebook to load environment variables and imports
%run "environment_setup.ipynb"

# Initialize the SentenceTransformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Define an embedding function with batching
class SentenceTransformerEmbeddings:
    def __init__(self, model, batch_size=2):
        self.model = model
        self.batch_size = batch_size

    def embed_documents(self, documents):
        embeddings = []
        for start_idx in range(0, len(documents), self.batch_size):
            batch = documents[start_idx:start_idx + self.batch_size]
            batch_embeddings = self.model.encode(batch, show_progress_bar=True).tolist()
            embeddings.extend(batch_embeddings)
        return embeddings

    def embed_query(self, query):
        return self.model.encode(query, show_progress_bar=True).tolist()

    def __call__(self, text):
        return self.model.encode(text, show_progress_bar=True).tolist()

embedding_function = SentenceTransformerEmbeddings(model, batch_size=1)  # Batch size set to 2

# Define the ChunkingHelper class
class ChunkingHelper:
    def __init__(self, file_path: str, chunking_method: str, embedding_function):
        self.file_path = file_path
        self.chunking_method = chunking_method
        self.embedding_function = embedding_function
        self.loader = PyMuPDFLoader(file_path=file_path)
        self.docs = self.loader.load()
        self.chunks = []

    def chunk_document(self):
        if self.chunking_method == 'standard_deviation':
            text_splitter = SemanticChunker(self.embedding_function, breakpoint_threshold_type='standard_deviation')
        elif self.chunking_method == 'recursive_character':
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        else:
            raise ValueError(f"Unknown chunking method: {self.chunking_method}")
        
        self.chunks = text_splitter.split_documents(self.docs)

    def get_chunks(self):
        if not self.chunks:
            self.chunk_document()
        return self.chunks

# Paths to the directories containing the PDF files
directories = {
    'chroma': r"E:\RepoFisiereMulte\FisiereTotal",
    'faiss': r"E:\RepoFisiereMulte\FisiereTotal",
    'pinecone': r"E:\RepoFisiereMulte\FisiereTotal"
}

all_texts = {key: [] for key in directories}
all_metadatas = {key: [] for key in directories}

def process_pdfs(directory_path, all_texts, all_metadatas, chunking_method):
    for filename in os.listdir(directory_path):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory_path, filename)
            chunking_helper = ChunkingHelper(file_path, chunking_method, embedding_function)
            splits = chunking_helper.get_chunks()
            for i, split in enumerate(splits):
                split.metadata['chunk_id'] = i
                split.metadata['file'] = filename
                all_texts.append(split)
                all_metadatas.append(split.metadata)

# Process PDFs with the specified chunking method
chunking_method = 'recursive_character'  # or 'recursive_character'
for key, directory in directories.items():
    process_pdfs(directory, all_texts[key], all_metadatas[key], chunking_method)


AZURE_API_KEY: d5c1ae66fe5a41fd82f61e0b9fe88470
AZURE_API_VERSION: 2024-02-15-preview
AZURE_DEPLOYMENT: gpt-4t
AZURE_ENDPOINT: https://itp-azopenai-ne.openai.azure.com/


C:\Users\vlad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\vlad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
